In [9]:
import matplotlib.pyplot as plt
import skimage.io as io
import random
import sys
import os
import json
import numpy as np

from vqa_api.PythonEvaluationTools.vqaEvaluation.vqaEval import VQAEval
from vqa_api.PythonHelperTools.vqaTools.vqa import VQA
from q_type_prior import qTypePrior
from collections import Counter

NameError: name 'question_types' is not defined

In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
path_question_types = os.path.join(dataDir, "QuestionTypes")
with open(os.path.join(path_question_types, "mscoco_question_types.txt")) as f:
    question_types = f.read().splitlines()

d = {q_type:[annotation["answers"][i]["answer"] for i in range(10) for annotation in self.annotations
if annotation["question_type"] == q_type] for q_type in question_types}

self.top_answers = {q_type:Counter(d[q_type]).most_common(1)[0][0] for q_type in d.keys()}

In [4]:
#dataDir     = r'C:\Users\Nasser Benab\Documents\git\data\vqa'
dataDir     = r'/Users/adib/Documents/Cours 3A/Projet 3A/OR/VQA/vqa_dataset'
versionType = '' 
taskType    = 'OpenEnded' 
dataType    = 'mscoco'
dataSubTypes = ['train2014']

In [ ]:
# First baseline method: random answer
qt = qTypePrior(dataDir, versionType, taskType, dataType, dataSubTypes)

In [ ]:
# Predict answers on the validation dataset
dataSubType = "val2014"
qt.predict(dataSubType)

In [ ]:
# Name of the baseline method used
method_name = qt.__class__.__name__

fileTypes   = ['results', 'accuracy', 'evalQA', 'evalQuesType', 'evalAnsType']

[resFile, accuracyFile, evalQAFile, evalQuesTypeFile, evalAnsTypeFile] = \
['%s/Results/%s%s_%s_%s_%s_%s.json'%(dataDir, versionType, taskType, dataType, dataSubType, \
method_name, fileType) for fileType in fileTypes] 
print("{} results file:".format(method_name), resFile)

In [ ]:
# vqa object for the test 
vqa = qt.vqa_test
# Create vqaRes object
vqaRes = vqa.loadRes(resFile, qt.quesFile_test)

In [ ]:
# Create vqaEval object by taking vqa and vqaRes
vqaEval = VQAEval(vqa, vqaRes, n=2)  

In [ ]:
# Evaluate results
vqaEval.evaluate() 

# Print accuracies
print("\n")
print("Overall Accuracy is: %.02f\n" %(vqaEval.accuracy['overall']))
print("Per Question Type Accuracy is the following:")
for quesType in vqaEval.accuracy['perQuestionType']:
    print("%s : %.02f" %(quesType, vqaEval.accuracy['perQuestionType'][quesType]))
print("\n")
print("Per Answer Type Accuracy is the following:")
for ansType in vqaEval.accuracy['perAnswerType']:
    print("%s : %.02f" %(ansType, vqaEval.accuracy['perAnswerType'][ansType]))
print("\n")

# demo how to use evalQA to retrieve low score result
# 35 is per question percentage accuracy
# evals = [quesId for quesId in vqaEval.evalQA if vqaEval.evalQA[quesId]<35]   
# if len(evals) > 0:
#     print('ground truth answers')
#     randomEval = random.choice(evals)
#     randomAnn = vqa.loadQA(randomEval)
#     vqa.showQA(randomAnn)

#     print('\n')
#     print('generated answer (accuracy %.02f)'%(vqaEval.evalQA[randomEval]))
#     ann = vqaRes.loadQA(randomEval)[0]
#     print("Answer:   %s\n" %(ann['answer']))

#     imgId = randomAnn[0]['image_id']
#     imgFilename = 'COCO_' + dataSubType + '_'+ str(imgId).zfill(12) + '.jpg'
#     if os.path.isfile(imgDir + imgFilename):
#         I = io.imread(imgDir + imgFilename)
#         plt.imshow(I)
#         plt.axis('off')
#         plt.show()

# # plot accuracy for various question types
# plt.bar(list(range(len(vqaEval.accuracy['perQuestionType']))), 
#         list(vqaEval.accuracy['perQuestionType'].values()), align='center')
# plt.xticks(list(range(len(vqaEval.accuracy['perQuestionType']))), 
#            list(vqaEval.accuracy['perQuestionType'].keys()), 
#            rotation='0',fontsize=10)
# plt.title('Per Question Type Accuracy', fontsize=10)
# plt.xlabel('Question Types', fontsize=10)
# plt.ylabel('Accuracy', fontsize=10)
# plt.show()

# save evaluation results to ./Results folder
# json.dump(vqaEval.accuracy,     open(accuracyFile,     'w'))
# json.dump(vqaEval.evalQA,       open(evalQAFile,       'w'))
# json.dump(vqaEval.evalQuesType, open(evalQuesTypeFile, 'w'))
# json.dump(vqaEval.evalAnsType,  open(evalAnsTypeFile,  'w'))